# Assignment 2
###### Team Members: 
Katelyn Vincent (8:30am)  
Jenny Robinson (10:30am)  
Jordan Pflum (10:30am)  
Bryant Leal (10:30am)  
Kessiena Ofunrein (8:30am)

# Task A - Web Scraper

In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
from collections import namedtuple
import requests
import re
import pandas as pd
import threading

from selenium import webdriver                      
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
#Getting top 250 URLS from beautiful soup
def scrape_beer_urls(page_url):
    """
    This function gets the urls from a list of beer urls and puts them in a list.
    """
    page_response = requests.get(page_url, timeout=5)

    full_soup = BeautifulSoup(page_response.content, 'lxml')

    beer_table = full_soup.find('div', id='ba-content').table

    def is_a_and_parent_is_td(tag):
        return tag.parent.name == 'td' and tag.name == 'a'

    all_beers = beer_table.find_all(is_a_and_parent_is_td)
    beer_urls = []

    for beer_tag in all_beers:
        beer_urls.append('https://www.beeradvocate.com' + beer_tag.get('href'))
        print('https://www.beeradvocate.com' + beer_tag.get('href'))

    return beer_urls

In [ ]:
#testing
url = 'https://www.beeradvocate.com/beer/top-rated/'
beer_urls = scrape_beer_urls(url)

In [ ]:
# Source: https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab
driver = webdriver.Chrome('C:\chromedriver\chromedriver')
driver.set_page_load_timeout(60)


com = [] 
beer_name_list = []
rate_list = []

for url in beer_urls[0:250]:
    '''
    getting names of beers
    
    '''
    page_response = requests.get(url, timeout=5)
    full_soup = BeautifulSoup(page_response.content, 'lxml')
    title = full_soup.find('div', class_='titleBar')
    beer_name = list(title.h1.strings)[0].strip()
    
    '''
    Getting rating out of each reviewer
    '''
    rating_path = full_soup.find_all('span', class_='BAscore_norm')   
    '''
    Getting comments out of each beer section
    '''
    driver.get(url)
    comment_element= driver.find_elements(By.XPATH,'//*[@id="rating_fullview_content_2"]')
    for i in range(len(comment_element)):
        com.append(comment_element[i].text)
        beer_name_list.append(beer_name)
        rate_list.append(rating_path[i].get_text())

print(beer_name_list)
print(rate_list)
print(com)
    #for comment in range(len(userid_element)):
     #   user_id.append(userid_element[comment].text)
#print(user_id)
driver.close()

In [ ]:
# cleaning
com_split = []
for i in range(len(com)):
    com_split.append(com[i].split('\n\n'))
    
clean_comments=[]
for i in range(len(com_split)):
    clean_comments.append(''.join(com_split[i][2:-1]))
    
cleanest_comments = []
for i in range(len(clean_comments)):
    cleanest_comments.append(clean_comments[i].replace('\n',' '))

In [ ]:
# compiling dataframe
df = pd.DataFrame(columns=['product_name','reviews', 'ratings'])
df['reviews'] = cleanest_comments
df['product_name']= beer_name_list
df['ratings'] = rate_list

In [ ]:
# convert to csv
df.to_csv('beer_comments.csv')

# Task B - Word Frequency

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [3]:
df = pd.read_csv('beer_comments.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.rename(columns={"reviews": "product_review", "ratings": "user_rating"}, inplace = True)
df.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,Smell: early morning pancakes and coffee befor...,5.00
1,Kentucky Brunch Brand Stout,2019 vintage. Pours a very dark brown color wi...,4.53
2,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doin...,1.49
3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage. This pours thick and c...,4.52
4,Kentucky Brunch Brand Stout,2018 version. Poured dark with a small head. S...,4.99


In [4]:
def get_word_frequency(wordlist):
    # replace slash with space
    wordlist = wordlist.replace("/", " ")
    wordlist = wordlist.replace("_", " ")
    wordlist = wordlist.replace("'", " ")
    wordlist = wordlist.replace("-", " ")

    # tokenize word
    tokens = nltk.tokenize.word_tokenize(wordlist)

    # Get list of English stop words
    take_out = stopwords.words('english')
    take_out = [word.upper() for word in take_out]

    # Convert word to uppercase
    tokens = [word.upper() for word in tokens]

    # Filter out stop words and punctuation from tokens list
    tokens = [word for word in tokens if word not in take_out]
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word[0] not in string.punctuation]

    # Get word frequency distribution
    word_frequencies = nltk.FreqDist(tokens)
 
    # Sort word frequency distribution by number of times each word occurs
    sorted_counts = sorted(word_frequencies.items() , key = lambda x: x[1] ,
                           reverse = True)
 
    return sorted_counts

In [5]:
def get_word_frequency_all_comments(webScraperData):
    # convert comments column to strings
    webScraperData['product_review'] = webScraperData['product_review'].apply(str)
    # create a list of the comments
    comments = webScraperData['product_review'].tolist()

    preresults = [x for x in comments if x != '']
    results = [get_word_frequency(x) for x in preresults]

    # get list of unique words in posts
    allwords = []
    for comment in results:
        for word in comment:
            # Append Word form (Word, Occurance) tuple
            allwords.append(word[0])

    # get word frequency across all comments (counting each word only once per post)
    overall_text = ' '.join(allwords)
    top_words = get_word_frequency(overall_text)
    overall_freq = pd.DataFrame(top_words,columns=['word','count'])
    
    return overall_freq

In [6]:
get_word_frequency_all_comments(df)[:10]

,word,count
0,HEAD,3225
1,BEER,2728
2,TASTE,2147
3,LIKE,1693
4,SWEET,1685
5,DARK,1669
6,ONE,1602
7,CHOCOLATE,1548
8,NOSE,1486
9,WELL,1463


# Task C - Cosine Similarity

In [7]:
# import libraries
import numpy as np
import pandas as pd
import string
from pandas import DataFrame
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
# create list of top three attributes
attributes = ['head taste sweet']

# combine attributes and reviews
documents = attributes + df['product_review'].tolist()
documents = ([str(comment) for comment in documents])

# clean reviews by removing stopwords and punctuation
def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

cleaned = list(map(clean_string, documents))

# apply vectorizer to list of attributes and reviews, and convert to array
vectorizer = CountVectorizer().fit_transform(cleaned)
vectors = vectorizer.toarray()

In [9]:
# calculate cosine similarity between attributes and each review
def cosine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

# create list of cosine similarities, and add to df
cosine_sims = []
for i in range(1,len(vectors)):
    cosine_sims.append(cosine_sim_vectors(vectors[0], vectors[i]))
    
df['cosine_similarity_score'] = cosine_sims
df.head()

,product_name,product_review,user_rating,cosine_similarity_score
0,Kentucky Brunch Brand Stout,Smell: early morning pancakes and coffee befor...,5.00,0.100504
1,Kentucky Brunch Brand Stout,2019 vintage. Pours a very dark brown color wi...,4.53,0.000000
2,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doin...,1.49,0.000000
3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage. This pours thick and c...,4.52,0.123797
4,Kentucky Brunch Brand Stout,2018 version. Poured dark with a small head. S...,4.99,0.210819


# Task D - Sentiment Analysis

In [10]:
import warnings
import pandas as pd 
import re
import string
from more_itertools import locate
import nltk
from nltk.corpus import stopwords
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
warnings.filterwarnings("ignore")

In [11]:
#READ DATA
df_uncleaned = pd.read_csv("beer_comments.csv")
df_sentiment = df_uncleaned.dropna()

In [12]:
#SEARCH FOR "N" WORDS NEAR STOP WORD 
def search(text,n,search_word):
    import re
    #'''Searches for text, and retrieves n words either side of the text, which are retuned seperatly'''
    word = r"\W*([\w]+)"
    groups = re.search(r'{}\W*{}{}'.format(word*n,search_word,word*n), text).groups()
    return groups[:n],groups[n:]

def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

In [13]:
def tokenize(wordlist):
    # replace slash with space
    wordlist = wordlist.replace("/", " ")
    wordlist = wordlist.replace("_", " ")
    wordlist = wordlist.replace("'", " ")
    wordlist = wordlist.replace("-", " ")
    # tokenize word
    tokens = nltk.tokenize.word_tokenize(wordlist)
    # Get list of English stop words
    take_out = stopwords.words('english')
    take_out = [word.lower() for word in take_out]
    tokens = [word for word in tokens if word.lower() not in take_out]
    tokens = [word for word in tokens if word.lower() not in string.punctuation]
    tokens = [word for word in tokens if word[0].lower() not in string.punctuation]
    return tokens

def all_feat_mentions(review):
    list_of_matches = []
    split = tokenize(review)
    matches = list(locate(split, lambda x: x==feat))
    for i in matches:
        list_of_matches.append(split[i-1])
        list_of_matches.append(split[i+1])
    matches_list = ' '.join(list_of_matches)
    return matches_list

def f_sentiment(review):
    if (contains_word(review,feat)):
        reviewWOpunctuation = review.translate(str.maketrans('', '', string.punctuation))
        rev_check = reviewWOpunctuation.split()
        rev_check = tokenize(review)
        if (rev_check[-1] == feat):
            # If Feature is the last word, grab last three words
            trigram = rev_check[:-3]
            feat_gram = ' '.join(trigram)
            sentiment = vader.polarity_scores(feat_gram)['compound']
        elif (rev_check[0] == feat):
            # If Feature is the first word, grab first three words
            trigram = rev_check[:3]
            feat_gram = ' '.join(trigram)
            sentiment = vader.polarity_scores(feat_gram)['compound']
        else: #(rev_check[0]!=feat or rev_check[-1]!=feat):
            feat_gram = all_feat_mentions(review)
            sentiment = vader.polarity_scores(feat_gram)['compound']
        #else:
            #feat_gram = ' '
            #trigram = search(review,1,feat)[0]
            #feat_gram = ' '.join(trigram)
            #sentiment = vader.polarity_scores(feat_gram)['compound']
    else:
        sentiment = 0
    return sentiment

In [14]:
#APPLY FUNCTION TO EVERY FEATURE
features = ['taste','sweet','head']
for feat in features: 
    df_sentiment[feat] = df_sentiment['reviews'].apply(f_sentiment)
    
#AVG SENTIMENT SCORE
df_sentiment['avg_sentiment'] = (df_sentiment.iloc[:,-1] + df_sentiment.iloc[:,-2] + df_sentiment.iloc[:,-3]) / 3
df_sentiment.head()

,Unnamed: 0,product_name,reviews,ratings,taste,sweet,head,avg_sentiment
0,0,Kentucky Brunch Brand Stout,Smell: early morning pancakes and coffee befor...,5.00,0.0000,0.0,0.0000,0.0000
1,1,Kentucky Brunch Brand Stout,2019 vintage. Pours a very dark brown color wi...,4.53,0.0000,0.0,0.0000,0.0000
2,2,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doin...,1.49,0.0000,0.0,0.0000,0.0000
3,3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage. This pours thick and c...,4.52,-0.3612,0.0,-0.4767,-0.2793
4,4,Kentucky Brunch Brand Stout,2018 version. Poured dark with a small head. S...,4.99,0.5859,0.0,0.0000,0.1953


# Task E - Evaluation Score

In [15]:
def reccomendProduct(sentimentAnalysisData, similarityAnalysisData):
    '''
    Reccomend a products based on evaluation score (avg sentiment for product and similiarity score)
    :param sentimentAnalysisData: dataframe that provides the avg sentiment for each product
    :param similarityAnalysisData: dataframe that provides the similarity between a comment for a prouct and the arrtibutes
    '''
    # Calculate average similarity for each product
    avg_cosine_sim = similarityAnalysisData.groupby('product_name')['cosine_similarity_score'].mean().reset_index()
    
    # Calculate Evaluation score for every product
    productReccomentCriteria = sentimentAnalysisData.merge(avg_cosine_sim, how='inner', on='product_name')
    productReccomentCriteria['evaluation_score'] = productReccomentCriteria['avg_sentiment'] + productReccomentCriteria['cosine_similarity_score']
    
    return productReccomentCriteria[['product_name', 'avg_sentiment', 'cosine_similarity_score', 'evaluation_score']].sort_values('evaluation_score', ascending=False)

In [16]:
reccomendProduct(df_sentiment, df).head(3)

,product_name,avg_sentiment,cosine_similarity_score,evaluation_score
2795,Double Shot,0.508100,0.116340,0.624440
802,Morning Wood,0.438967,0.127617,0.566584
4953,Ten FIDY - Bourbon Barrel Aged,0.420667,0.132602,0.553269


The top three products we recommend based on highest cosine/sentiment evaluation score are Double Shot, Morning Wood, and Ten FIDY- Bourbon Barrel Aged.

# Task F - Spacy Similarity

In [18]:
import spacy
nlp = spacy.load(r'D:\anaconda3\Lib\site-packages\spacy\en_core_web_sm\en_core_web_sm-2.3.1')

In [19]:
# create list of spacy similarities, and add to df
spacy_sims = []
for i in range(1,len(cleaned)):
    doc1 = nlp(cleaned[0])
    doc2 = nlp(cleaned[i])
    spacy_sims.append(doc1.similarity(doc2))

df['spacy_similarity_score'] = spacy_sims

In [20]:
def reccomendProductSpacy(sentimentAnalysisData, similarityAnalysisData):
    '''
    Reccomend a products based on evaluation score (avg sentiment for product and similiarity score)
    :param sentimentAnalysisData: dataframe that provides the avg sentiment for each product
    :param similarityAnalysisData: dataframe that provides the similarity between a comment for a prouct and the arrtibutes
    '''
    # Calculate average similarity for each product
    avg_spacy_sim = similarityAnalysisData.groupby('product_name')['spacy_similarity_score'].mean().reset_index()
    
    # Calculate Evaluation score for every product
    productReccomentCriteria = sentimentAnalysisData.merge(avg_spacy_sim, how='inner', on='product_name')
    productReccomentCriteria['spacy_evaluation_score'] = productReccomentCriteria['avg_sentiment'] + productReccomentCriteria['spacy_similarity_score']
    
    return productReccomentCriteria[['product_name', 'avg_sentiment', 'spacy_similarity_score', 'spacy_evaluation_score']].sort_values('spacy_evaluation_score', ascending=False)

In [21]:
reccomendProductSpacy(df_sentiment, df).head(3)

,product_name,avg_sentiment,spacy_similarity_score,spacy_evaluation_score
2795,Double Shot,0.508100,0.671946,1.180046
802,Morning Wood,0.438967,0.676024,1.114990
5595,Space Trace,0.436233,0.670650,1.106883


The top three products we recommend based on highest spacy/sentiment evaluation score are Double Shot, The Abyss, and Morning Wood.

In [22]:
# calculate % attribute mentions
import copy
df2 = copy.deepcopy(df_sentiment)
df2["reviews"] = df2["reviews"].str.lower()
df2.head()

df2["head_count"] = df2["reviews"].map(lambda x: 1 if "head" in x else 0)
df2["taste_count"] = df2["reviews"].map(lambda x: 1 if "taste" in x else 0)
df2["sweet_count"] = df2["reviews"].map(lambda x: 1 if "sweet" in x else 0)
df2["att_count"] = df2[["head_count", "taste_count", "sweet_count"]].max(axis=1)
df2["review_count"] = 1

df_sent2 = df2.groupby('product_name').sum().reset_index()
df_sent2["%att_mentions"] = df_sent2['att_count'] / df_sent2['review_count']
df_sent2.drop(['Unnamed: 0','head_count','taste_count','sweet_count','review_count'], axis = 1, inplace = True)

df_sent2.sort_values(by='%att_mentions', ascending= False).head()

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
170,Moment Of Clarity,109.85,0.0000,1.8856,0.7586,0.881400,25,1.0
141,Jam The Radar,113.86,0.4588,0.9778,0.0000,0.478867,25,1.0
7,Abraxas,112.45,-0.4404,1.5460,1.4711,0.858900,25,1.0
95,Emerald Grouper,105.02,0.4404,0.0373,1.5783,0.685333,25,1.0
178,No Rules,108.29,0.2230,0.4508,0.4019,0.358567,25,1.0


In [23]:
df_sent2[df_sent2['product_name'] == 'Double Shot']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
89,Double Shot,109.26,0.3612,0.6536,0.7742,0.596333,21,0.84


In [24]:
df_sent2[df_sent2['product_name'] == 'Morning Wood']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
173,Morning Wood,112.02,0.0826,2.0621,0.4404,0.8617,22,0.88


In [25]:
df_sent2[df_sent2['product_name'] == 'Ten FIDY - Bourbon Barrel Aged']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
222,Ten FIDY - Bourbon Barrel Aged,106.32,1.0862,1.3435,1.2645,1.2314,23,0.958333


In [26]:
df_sent2[df_sent2['product_name'] == 'The Abyss']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
223,The Abyss,111.06,1.7545,0.0,1.2243,0.992933,22,0.88


Interestingly, two of the top three products for the cosine and spacy evalation scores were the same - Double Shot and Morning Wood.  84% of the reviews for Double Shot mentioned one of the three attributes (head, taste, and sweet) while 88% of reviews for Morning Wood included mention of one of the three attributes.  The third highest recommendation based on cosine evaluation score was Ten FIDY - Bourbon Barrel Aged, which had over 95% of reviews with mentions of the important attributes.  On the other hand, the third highest product recommendation from the spacy evaluation score was The Abyss with 88% of reviews including mentions of the important attributes. Since two of the top three recommendations were the same, it seems that bag-of-word and pretrained word vectors performed similarly in this particular analysis.

# Task G

In [27]:
# group by products
df_products = df.groupby('product_name').mean().reset_index()

In [28]:
# top 3 products by user_rating
df_products.sort_values(by='user_rating', ascending=False)[:3]

,product_name,user_rating,cosine_similarity_score,spacy_similarity_score
56,Chemtrailmix,4.754286,0.071085,0.675810
139,It Was All A Dream,4.747200,0.104242,0.653060
196,SR-71,4.746400,0.056189,0.645574


In [29]:
df_sent2[df_sent2['product_name'] == 'Chemtrailmix']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
56,Chemtrailmix,99.84,0.6369,0.0,0.0,0.2123,14,0.666667


In [30]:
df_sent2[df_sent2['product_name'] == 'It Was All A Dream']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
139,It Was All A Dream,118.68,0.0,0.4939,2.0087,0.8342,18,0.72


In [31]:
df_sent2[df_sent2['product_name'] == 'SR-71']

,product_name,ratings,taste,sweet,head,avg_sentiment,att_count,%att_mentions
196,SR-71,118.66,0.0,1.5619,0.0,0.520633,14,0.56


The top three products we recommend based on highest average user rating are Chemtrailmix, It Was All A Dream, and SR-71.  All three of these had very low cosine similarity scores, but relatively high spacy similarity scores.  The average sentiment scores for the three are across the board - for example, Chemtrailmix (the beer with the highest average user rating) has an average sentiment score of .21, which is very low.  The second highest, It Was All A Dream, is higher at .83, and the third highest beer in terms of user rating had an average sentiment score of .52.  It does not seem that all of these products would meet the requirements of our user since the similarity scores are not very high for the attributes that are important to them.